In [7]:
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
from detectree2.models.predict import predict_on_data
from detectree2.models.train import setup_cfg
from detectron2.engine import DefaultPredictor
import rasterio

In [8]:
# Path to site folder and orthomosaic
site_path = "/mnt/d_drive/detectree2/griffith_site"
img_path = site_path + "/rgb/young_2.tif"
tiles_path = site_path + "/tilespred/"

# Location of trained model
model_path = "/mnt/d_drive/detectree2/griffith_site/models/3site_model.pth"

# Specify tiling
buffer = 30
tile_width = 40
tile_height = 40
tile_data(img_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

INFO:detectree2.preprocessing.tiling:Tiling complete


In [9]:
trained_model = model_path
cfg = setup_cfg(update_model=trained_model)
predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))

INFO:detectron2.checkpoint.detection_checkpoint:[DetectionCheckpointer] Loading from /mnt/d_drive/detectree2/griffith_site/models/3site_model.pth ...
INFO:fvcore.common.checkpoint:[Checkpointer] Loading from /mnt/d_drive/detectree2/griffith_site/models/3site_model.pth ...
/home/ubuntu/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via

Predicting 6 files


In [10]:
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")

Projecting 6 files


In [11]:
crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)
clean = clean_crowns(crowns, 0.6, confidence=0)

Cleaning 233 crowns
0 / 233 crowns cleaned


In [12]:
clean.to_file(site_path + "/young2_crowns_out.gpkg")

INFO:pyogrio._io:Created 231 records


In [11]:
from utils import scp_to_ec2
source=r"C:\Users\RoozbehKoochak\Downloads\2016_Griffith_visual_cropped_410114_6211532_40_30_7855.png"
dest='/mnt/d_drive/detectree2/griffith_site/rgb'
scp_to_ec2('3.25.210.234',source=source,destination=dest)

'scp -i "C:\\Users\\RoozbehKoochak\\.ssh\\Rooz_server_key.pem" "C:\\Users\\RoozbehKoochak\\Downloads\\2016_Griffith_visual_cropped_410114_6211532_40_30_7855.png" ubuntu@3.25.210.234:"/mnt/d_drive/detectree2/griffith_site/rgb"'

In [8]:
from utils import scp_to_local
source='/mnt/d_drive/detectree2/griffith_site/rgb/2016_Cowra_visual_cropped_670111_6271451_40_30_7855.tif'
dest=r"C:\Users\RoozbehKoochak\Downloads"
scp_to_local('3.25.210.234',source=source,destination=dest)

'scp -i "C:\\Users\\RoozbehKoochak\\.ssh\\Rooz_server_key.pem" ubuntu@3.25.210.234:"/mnt/d_drive/detectree2/griffith_site/rgb/2016_Cowra_visual_cropped_670111_6271451_40_30_7855.tif" "C:\\Users\\RoozbehKoochak\\Downloads"'